In [2]:
import pkg_resources
import sys

# 定义「必装包 + 适配版本」（解决版本兼容问题）
# 格式：{"包名": "适配版本"}
required_packages = {
    "streamlit": "1.52.2",
    "langchain": "0.1.17",  # 稳定版，无需langchain-openai
    "langchain-community": "0.0.36",
    "langchain-openai": "0.2.7",  # 可选（0.1.x无需安装）
    "chromadb": "0.5.17",
    "python-dotenv": "1.2.1",
    "pypdf": "6.4.0",
    "jieba": "0.42.1"
}

print("=" * 60)
print("Python 依赖包检查结果 + 安装命令生成")
print("=" * 60)

# 1. 检查现有包状态
missing_packages = []  # 未安装的包
error_packages = []  # 安装但导入失败的包
ok_packages = []  # 正常的包

for pkg_name, target_version in required_packages.items():
    try:
        # 处理包名转换（langchain-openai → langchain_openai）
        import_name = pkg_name.replace("-", "_") if "-" in pkg_name else pkg_name

        # 获取已安装版本
        installed_version = pkg_resources.get_distribution(pkg_name).version

        # 验证导入
        __import__(import_name)

        # 版本检查（提示版本不一致）
        if installed_version != target_version:
            print(f"⚠️ {pkg_name:<20} 已安装(版本不符) 安装版: {installed_version} | 推荐版: {target_version}")
            missing_packages.append(f"{pkg_name}=={target_version}")  # 加入重装列表
        else:
            print(f"✅ {pkg_name:<20} 已安装(版本正确) 版本: {installed_version}")
            ok_packages.append(pkg_name)

    except pkg_resources.DistributionNotFound:
        print(f"❌ {pkg_name:<20} 未安装")
        missing_packages.append(f"{pkg_name}=={target_version}")  # 加入安装列表

    except ImportError as e:
        print(f"❌ {pkg_name:<20} 导入失败 版本: {installed_version} | 错误: {str(e)[:40]}...")
        error_packages.append(f"{pkg_name}=={target_version}")  # 加入重装列表

    except Exception as e:
        print(f"❓ {pkg_name:<20} 检查异常: {str(e)[:40]}...")
        missing_packages.append(f"{pkg_name}=={target_version}")  # 加入重装列表

# 2. 生成安装命令
print("\n" + "=" * 60)
print("需要安装/重装的包列表")
print("=" * 60)

if missing_packages or error_packages:
    # 合并需要安装/重装的包
    all_need_install = list(set(missing_packages + error_packages))

    # 生成pip安装命令（区分Windows/Linux）
    pip_command = f"pip install {' '.join(all_need_install)} --force-reinstall"
    pip3_command = f"pip3 install {' '.join(all_need_install)} --force-reinstall"  # 备用

    print(f"\n📝 一键安装命令（复制执行）:")
    print(f"Windows: {pip_command}")
    print(f"Linux/Mac: {pip3_command}")

    # 单独处理langchain-openai（0.1.x版本无需安装）
    if "langchain==0.1.17" in missing_packages and "langchain-openai==0.2.7" in missing_packages:
        print("\n⚠️ 重要提示:")
        print("  LangChain 0.1.17 版本无需安装 langchain-openai，可执行以下精简命令:")
        print(
            f"  pip install langchain==0.1.17 langchain-community==0.0.36 chromadb==0.5.17 python-dotenv==1.2.1 streamlit==1.52.2 pypdf==6.4.0 jieba==0.42.1 --force-reinstall")
else:
    print("✅ 所有依赖包已正确安装（版本匹配），无需额外安装！")

# 3. 打印Python环境信息
print("\n" + "=" * 60)
print("Python 环境信息")
print("=" * 60)
print(f"Python 版本: {sys.version.split()[0]}")
print(f"Python 解释器路径: {sys.executable}")
print(f"pip 命令建议: {'pip' if 'python.exe' in sys.executable else 'pip3'}")
print("=" * 60)

Python 依赖包检查结果 + 安装命令生成
✅ streamlit            已安装(版本正确) 版本: 1.52.2
⚠️ langchain            已安装(版本不符) 安装版: 0.4.0.dev0 | 推荐版: 0.1.17
⚠️ langchain-community  已安装(版本不符) 安装版: 0.4.1 | 推荐版: 0.0.36
⚠️ langchain-openai     已安装(版本不符) 安装版: 0.2.6 | 推荐版: 0.2.7
✅ chromadb             已安装(版本正确) 版本: 0.5.17
❌ python-dotenv        导入失败 版本: 1.2.1 | 错误: No module named 'python_dotenv'...
✅ pypdf                已安装(版本正确) 版本: 6.4.0
✅ jieba                已安装(版本正确) 版本: 0.42.1

需要安装/重装的包列表

📝 一键安装命令（复制执行）:
Windows: pip install langchain-openai==0.2.7 langchain==0.1.17 python-dotenv==1.2.1 langchain-community==0.0.36 --force-reinstall
Linux/Mac: pip3 install langchain-openai==0.2.7 langchain==0.1.17 python-dotenv==1.2.1 langchain-community==0.0.36 --force-reinstall

⚠️ 重要提示:
  LangChain 0.1.17 版本无需安装 langchain-openai，可执行以下精简命令:
  pip install langchain==0.1.17 langchain-community==0.0.36 chromadb==0.5.17 python-dotenv==1.2.1 streamlit==1.52.2 pypdf==6.4.0 jieba==0.42.1 --force-reinstall

Python 环境信息
Python 版本:

In [4]:
doc_path = "./docs/疾病.txt"
from langchain.schema import Document
def load_documents(filename: str):
    """
    加载文档并转换为LangChain的Document格式
    :param filename: 文档路径（支持txt格式）
    :return: Document对象列表
    """
    documents = []
    try:
        with open(doc_path, 'r', encoding='utf-8') as f:
            doc_text = f.read()

        # 封装为Document对象（可添加元数据，如文档名称、路径等）
        document = Document(
            page_content=doc_text,
            metadata={"source": doc_path}
        )
        documents.append(document)
        print(f"成功加载文档：{doc_path}，文本长度：{len(doc_text)}字符")
    except Exception as e:
        print(f"加载文档失败：{e}")
    return documents
raw_docs = load_documents(doc_path)


成功加载文档：./docs/疾病.txt，文本长度：13611字符
